In [ ]:
!pip install elasticsearch==8.10.0


Found existing installation: elasticsearch 9.0.2
Uninstalling elasticsearch-9.0.2:
  Successfully uninstalled elasticsearch-9.0.2
   ---------------------------------------- 0.0/415.0 kB ? eta -:--:--
   ---- ---------------------------------- 51.2/415.0 kB 890.4 kB/s eta 0:00:01
   --------- ------------------------------ 102.4/415.0 kB 1.2 MB/s eta 0:00:01
   ------------- ------------------------ 143.4/415.0 kB 950.9 kB/s eta 0:00:01
   ----------------- -------------------- 194.6/415.0 kB 985.7 kB/s eta 0:00:01
   -------------------- ----------------- 225.3/415.0 kB 860.2 kB/s eta 0:00:01
   ----------------------- -------------- 256.0/415.0 kB 830.3 kB/s eta 0:00:01
   ------------------------------------- -- 389.1/415.0 kB 1.0 MB/s eta 0:00:01
   -------------------------------------- 415.0/415.0 kB 996.9 kB/s eta 0:00:00



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
!pip install requests


   ---------------------------------------- 0/2 [charset_normalizer]
   ---------------------------------------- 0/2 [charset_normalizer]
   ---------------------------------------- 0/2 [charset_normalizer]
   ---------------------------------------- 0/2 [charset_normalizer]
   -------------------- ------------------- 1/2 [requests]
   -------------------- ------------------- 1/2 [requests]
   -------------------- ------------------- 1/2 [requests]
   -------------------- ------------------- 1/2 [requests]
   -------------------- ------------------- 1/2 [requests]
   -------------------- ------------------- 1/2 [requests]
   ---------------------------------------- 2/2 [requests]



In [1]:
from elasticsearch import Elasticsearch

In [2]:
# es_client = Elasticsearch('http://localhost:9200') 
es_client = Elasticsearch(
    "http://localhost:9200",
    headers={
        "Accept": "application/vnd.elasticsearch+json; compatible-with=8",
        "Content-Type": "application/vnd.elasticsearch+json; compatible-with=8"
    }
)


In [3]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [5]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "question": {"type": "text"},
        }
    }
}

index_name = "course-questions-homework"

es_client.indices.create(index=index_name, body=index_settings)


C:\Users\LENOVO\AppData\Local\Temp\ipykernel_22404\2274343306.py:16: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  es_client.indices.create(index=index_name, body=index_settings)


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions-homework'})

In [6]:
for doc in documents:
    es_client.index(index=index_name, document=doc)

In [7]:
query = "How do execute a command on a Kubernetes pod?"

In [17]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text", ],
                        "type": "best_fields"
                    }
                },
               
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs, response

In [18]:
results, response = elastic_search(query)


C:\Users\LENOVO\AppData\Local\Temp\ipykernel_22404\1420315472.py:18: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  response = es_client.search(index=index_name, body=search_query)


In [19]:
results

[{'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I debug a docker container?',
  'course': 'machine-learning-zoomcamp'},
 {'text': 'Deploy and Access the Kubernetes Dashboard\nLuke',
  'section': '10. Kubernetes and TensorFlow Serving',
  'question': 'Kubernetes-dashboard',
  'course': 'machine-learning-zoomcamp'},
 {'text': 'You can copy files from your local machine into a Docker container using the docker cp command. Here\'s how to do it:\nIn the Dockerfile, you can provide the folder containing the files that you want to copy over. The basic syntax is as follows:\nCOPY ["src/predict.py", "models/xgb_model.bin", "./"]\t\t\t\t\t

In [20]:
response

ObjectApiResponse({'took': 132, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 739, 'relation': 'eq'}, 'max_score': 44.50556, 'hits': [{'_index': 'course-questions-homework', '_id': 'gUfacpcBknb7wcbRPaEs', '_score': 44.50556, '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)', 'section': '5. Deploying Machine Learning Models', 'question': 'How do I debug a docker container?', 'course': 'machine-learning-zoomcamp'}}, {'_index': 'course-questions-homework', '_id': 'EEfacpcBknb7wcbRe6Ke', '_score': 35.433445, '_source': {'text': 'Deploy and Access the Kubernetes Dashboard\nLuke', 'section': '10. Kubernetes and TensorFlow Serving', 

In [25]:
#the score of the top ranking result
response["hits"]["max_score"]

44.50556

In [79]:
def elastic_search(query):
    search_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course.keyword": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs


In [80]:
results = elastic_search("How do copy a file to a Docker container")

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_22404\3584162238.py:22: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  response = es_client.search(index=index_name, body=search_query)


In [81]:
results

[{'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I debug a docker container?',
  'course': 'machine-learning-zoomcamp'},
 {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\nHrithik Kumar Advani",
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I copy files from my local machine to docker container?',
 

In [82]:
#the 3rd question returned by the search engine
final_results[2]

{'text': 'You can copy files from your local machine into a Docker container using the docker cp command. Here\'s how to do it:\nIn the Dockerfile, you can provide the folder containing the files that you want to copy over. The basic syntax is as follows:\nCOPY ["src/predict.py", "models/xgb_model.bin", "./"]\t\t\t\t\t\t\t\t\t\t\tGopakumar Gopinathan',
 'section': '5. Deploying Machine Learning Models',
 'question': 'How do I copy files from a different folder into docker container’s working directory?',
 'course': 'machine-learning-zoomcamp'}

In [86]:
context_template = ""
for result in results:
    context_template =context_template+ f"""
    Q: {result["question"]}
    A: {result["text"]}
    """.strip()+"\n\n"

context_template

'Q: How do I debug a docker container?\n    A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)\n\nQ: How do I copy files from my local machine to docker container?\n    A: You can copy files from your local machine into a Docker container using the docker cp command. Here\'s how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\nHrithik Kumar Advani\n\nQ: How do I copy files from a different folder into docker container’s working directory?\n    A: You can copy files from your local machine into a Docker container using the dock

In [92]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

prompt = prompt_template.format(question="How do copy a file to a Docker container", context=context_template)

In [ ]:
#length of our prompt
print(len(prompt))

1459


In [94]:
!pip install tiktoken

   ---------------------------------------- 0.0/894.9 kB ? eta -:--:--
   ---------------------------------------- 0.0/894.9 kB ? eta -:--:--
   ----------------------- ---------------- 524.3/894.9 kB 2.1 MB/s eta 0:00:01
   ---------------------------------------- 894.9/894.9 kB 1.6 MB/s eta 0:00:00

   ---------------------------------------- 0/2 [regex]
   -------------------- ------------------- 1/2 [tiktoken]
   ---------------------------------------- 2/2 [tiktoken]



In [96]:
import tiktoken
encoding = tiktoken.encoding_for_model("gpt-4o")

In [99]:
tokens = encoding.encode(prompt)

# Count the number of tokens
num_tokens = len(tokens)


In [100]:
num_tokens

324